In [1]:
import json
import os
import re
import tensorflow as tf
import copy
import numpy as np
import matplotlib.pyplot as plt
import random

# Useful constants
INPUT = 'input'
OUTPUT = 'output'
CONV3X3 = 'conv3x3-bn-relu'
CONV1X1 = 'conv1x1-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'
NUM_VERTICES = 7
MAX_EDGES = 9
EDGE_SPOTS = NUM_VERTICES * (NUM_VERTICES - 1) / 2   # Upper triangular matrix
OP_SPOTS = NUM_VERTICES - 2   # Input/output vertices are fixed
ALLOWED_OPS = [CONV3X3, CONV1X1, MAXPOOL3X3]
ALLOWED_EDGES = [0, 1]   # Binary adjacency matrix

In [2]:
with tf.io.gfile.GFile('../results/vertices_2/evaluation/04/043721b9c7fe8c5fad811d47d83132ec/repeat_1/results.json') as f:
    result = json.load(f)
    print(result)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
count = 0
max_count = 423624 * 3

adjacency_list = []
operations_list = []

print('Generating dataset')
for serialized_row in tf.python_io.tf_record_iterator(dataset_file):

  count += 1
  print('\rCompleted: %0.2f%%' % (count/(max_count)*100), end='')

  # Take only unique adjacency matrices
  if count % 3 != 0: continue

  # Parse the data from the data file.
  module_hash, epochs, raw_adjacency, raw_operations, raw_metrics = (
      json.loads(serialized_row.decode('utf-8')))

  dim = int(np.sqrt(len(raw_adjacency)))
  adjacency = np.array([int(e) for e in list(raw_adjacency)], dtype=np.int8)
  adjacency = np.reshape(adjacency, (dim, dim))
  operations = raw_operations.split(',')
  metrics = model_metrics_pb2.ModelMetrics.FromString(
      base64.b64decode(raw_metrics))
  
  adjacency_list.append(adjacency)
  operations_list.append(operations)

  # Evaluation statistics at the end of training
  '''
  final_evaluation = metrics.evaluation_data[2]
  training_time = final_evaluation.training_time
  train_accuracy = final_evaluation.train_accuracy
  validation_accuracy = (
      final_evaluation.validation_accuracy)
  test_accuracy = final_evaluation.test_accuracy
  trainable_params = metrics.trainable_parameters

  print(f'Module {count//3+1}.{count%3+1} \nAdjacency matrix: \n{adjacency} \nOperations: {operations} \nTrainable parameters: {trainable_params}')
  print(f'Train Accuracy: {train_accuracy} \nValidation Accuracy: {validation_accuracy} \nTest Accuracy: {test_accuracy}\n')
  '''

Generating dataset
Completed: 100.00%

In [9]:
ordered_operations = [OUTPUT, INPUT, MAXPOOL3X3, CONV1X1, CONV3X3]

weighted_adjacency_list = copy.deepcopy(adjacency_list)
compressed_weighted_list = []

count = 0

print('Augmenting dataset')
for i in range(len(weighted_adjacency_list)):

  count += 1
  print('\rCompleted: %0.2f%%' % (count/(max_count/3)*100), end='')

  vertices = np.shape(weighted_adjacency_list[i])[0]

  for v in range(vertices):
    weighted_adjacency_list[i][v, :] *= (ordered_operations.index(operations_list[i][v]))

  compressed_weighted_list.append(compress(weighted_adjacency_list[i]))

Augmenting dataset
Completed: 100.00%

In [11]:
n = 10
print(adjacency_list[n])
print(operations_list[n])
print(weighted_adjacency_list[n])

[[0 1 1 1 0 0 0]
 [0 0 0 0 0 0 1]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 1]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0]]
['input', 'conv3x3-bn-relu', 'conv3x3-bn-relu', 'conv1x1-bn-relu', 'maxpool3x3', 'maxpool3x3', 'output']
[[0 1 1 1 0 0 0]
 [0 0 0 0 0 0 4]
 [0 0 0 4 0 0 0]
 [0 0 0 0 3 0 3]
 [0 0 0 0 0 2 0]
 [0 0 0 0 0 0 2]
 [0 0 0 0 0 0 0]]
